In [ ]:
!pip install pybamm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.6/22.6 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.3/72.3 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 3.5 MB/s eta 0:00:00


In [ ]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

options = {"thermal": "lumped"}
model = pybamm.lithium_ion.SPM(options=options)

parameter_values = pybamm.ParameterValues("Chen2020")

Current = [1, 2, 3, 4, 5]

# For SoC
SOC = 0.20    # For each stage

# Time for each stage using the formula: t = (SoC) * C / I
C = parameter_values["Nominal cell capacity [A.h]"]

# Initialize the lists to store the results for each cycle
data_all = []  # This will store all the data

# Create subplots for current, voltage, SoC, and temperature
fig, ax = plt.subplots(4, 1, figsize=(10, 20))
counter = 1
for I1 in Current:
    for I2 in Current:
        for I3 in Current:
            for I4 in Current:
                # Create temporary variables for the current values
                temp_I1 = I1
                temp_I2 = I2
                temp_I3 = I3
                temp_I4 = I4

                while True:  # Add a loop to keep trying until the temperature is under control
                    print(f'{counter}) Processing combination : {temp_I1}, {temp_I2}, {temp_I3}, {temp_I4}')
                    T1 = SOC * C / temp_I1  # Time for Stage 1 in hours
                    T2 = SOC * C / temp_I2  # Time for Stage 2 in hours
                    T3 = SOC * C / temp_I3  # Time for Stage 3 in hours
                    T4 = SOC * C / temp_I4  # Time for Stage 4 in hours

                    # Hours to minutes # *= multiplies the current value of the variable by a given value and then assigns the result back to the variable.
                    T1 *= 60
                    T2 *= 60
                    T3 *= 60
                    T4 *= 60

                    procedures = ([(
                        "Discharge at C/1 for 57 minutes",
                        f"Charge at {temp_I1}C for {T1} minutes or until 4.2 V",  # Stage 1
                        f"Charge at {temp_I2}C for {T2} minutes or until 4.2 V",  # Stage 2
                        f"Charge at {temp_I3}C for {T3} minutes or until 4.2 V",  # Stage 3
                        f"Charge at {temp_I4}C for {T4} minutes or until 4.2 V",  # Stage 4
                    )])

                    experiment = pybamm.Experiment(procedures)

                    sim = pybamm.Simulation(model, experiment=experiment, parameter_values=parameter_values)
                    solution = sim.solve()

                    discharge = 57 * 60
                    charging_time = sim.solution.t[sim.solution.t >= discharge]
                    currents = sim.solution["Current [A]"](charging_time)
                    voltages = sim.solution["Terminal voltage [V]"](charging_time)
                    capacity = solution["Discharge capacity [A.h]"].entries
                    SoC = (1 - capacity / C) * 100
                    charging_SoC = SoC[solution.t >= discharge]
                    temperatures = sim.solution["X-averaged cell temperature [C]"](charging_time)
                    offset = 2

                    # Check if temperature exceeds 45°C
                    if np.any(temperatures > 45):
                        print(f'Temperature exceeded 45°C for combination: {temp_I1}, {temp_I2}, {temp_I3}, {temp_I4}')
                        # Adjust the current by decreasing it with the step size
                        temp_I1 -= 0.1
                        temp_I2 -= 0.1
                        temp_I3 -= 0.1
                        temp_I4 -= 0.1
                        print(f'Adjusted current to: {temp_I1}, {temp_I2}, {temp_I3}, {temp_I4}')
                        continue
                    else:
                        break  # If the temperature is under control, break the while loop and proceed with the next combination

                counter += 1
                data_all.append({
                    'Currents': currents,
                    'Voltages': voltages,
                    'SoCs': charging_SoC,
                    'Temperatures': temperatures,
                    'Combination': f'{temp_I1}, {temp_I2}, {temp_I3}, {temp_I4}'
                })

                ax[0].plot(charging_time + offset, currents, label=f'Current {temp_I1}, {temp_I2}, {temp_I3}, {temp_I4}')
                ax[1].plot(charging_time, voltages, label=f'Current {temp_I1}, {temp_I2}, {temp_I3}, {temp_I4}')
                ax[2].plot(charging_time, charging_SoC, label=f'Current {temp_I1}, {temp_I2}, {temp_I3}, {temp_I4}')
                ax[3].plot(charging_time, temperatures, label=f'Current {temp_I1}, {temp_I2}, {temp_I3}, {temp_I4}')

ax[0].set_xlabel('Time [s]')
ax[0].set_ylabel('Current [A]')
#ax[0].legend()

ax[1].set_xlabel('Time [s]')
ax[1].set_ylabel('Voltage [V]')
#ax[1].legend()

ax[2].set_xlabel('Time [s]')
ax[2].set_ylabel('State of Charge [%]')
#ax[2].legend()

ax[3].set_xlabel('Time [s]')
ax[3].set_ylabel('Cell Temperature [C]')
#ax[3].legend()

plt.tight_layout()
plt.show()

df = pd.DataFrame(data_all)
df.to_excel('Test.xlsx')
